In [17]:
model = 'PreResNet110'
dataset = 'CIFAR10'
optimizer = 'sgd'
machine = 'jade2'
#choose either 'X' or 'W'
#Where 'X' is Xadam - AdamW+SWA
if optimizer =='adam':
    typeofadam = 'X'
if optimizer =='sgd':
    typeofsgd = ''


In [18]:
if machine == 'rapid':
    data_path = '/nfs/home/dgranziol/curvature/data/'
    save_path = '/nfs/home/dgranziol/kfac-curvature/out/'
elif machine == 'jade1':
    data_path = '/jmain01/home/JAD017/sjr01/dxg49-sjr01/curvature/data/'
    save_path = '/jmain01/home/JAD017/sjr01/dxg49-sjr01/kfac-curvature/out/'
elif machine == 'jade2':
    data_path = '/jmain01/home/JAD017/sjr02/dxg49-sjr02/curvature/data/'
    save_path = '/jmain01/home/JAD017/sjr02/dxg49-sjr02/kfac-curvature/out/'

start = 'python3 run_'+optimizer+'.py --dir '+save_path+' --dataset '+dataset+' --data_path='+data_path+ ' --model='+model+' --epochs=300 --save_freq=25 --eval_freq=1' 
        

In [19]:
#sgd_rate = [0.003,0.01,0.03,0.1,0.3]
sgd_rate = [0.001,0.005,0.01,0.05,]

#adam_rate = [0.0001,0.0003,0.01,0.03]
adam_rate = [0.00003,0.0001,0.0003,0.01]

wd = [0.00001,0.00005,0.0001,0.0005,0.001]
if optimizer =='sgd':
    gridval = sgd_rate
    appendname = typeofsgd
elif optimizer =='adam':
    gridval = adam_rate
    appendname = typeofadam
    

In [20]:
#full SPECTRUM
#eigenvectors not saved
if machine == 'jade1' or 'jade2':
    dirstart = 'jade_scripts/'
elif machine == 'rapid':
    dirstart = 'rapid_scripts/'
f = open(dirstart+"optim-"+optimizer+appendname+model+dataset+".sh","w+")
if machine =='jade1' or 'jade2':
        print('making it jade ready')
        f.write('#!/bin/bash')
        f.write('\n')
        f.write('# set the number of nodes')
        f.write('\n')
        f.write('#SBATCH --nodes=1')
        f.write('\n')
        f.write('#SBATCH --job-name='+optimizer+appendname)
        f.write('\n')
        f.write('#SBATCH --partition=small')
        f.write('\n')
        f.write('#SBATCH --gres=gpu:1')
        f.write('\n')
        f.write('module load python3/anaconda')
        f.write('\n')
        f.write('source activate diegorubin')
        f.write('\n')
for i in range(0,len(gridval)):
    for j in range(0,len(wd)):
        k = start+' --lr_init '+str(gridval[i])
        if optimizer =='adam':
            if typeofadam == 'W':
                k = k + ' --decoupled_wd'+' --wd='+str(wd[j]/gridval[i])
            elif typeofadam == 'X':
                k = k + ' --decoupled_wd'+' --wd='+str(wd[j]/gridval[i]) + ' --swag' +' --swag_lr='+str(gridval[i]/2)
        else:
            k = k +' --wd='+str(wd[j])
            if optimizer =='sgd':
                if typeofsgd == 'swa':
                    k = k + ' --swag' +' --swag_lr='+str(gridval[i]/2) 
        f.write(k)
        f.write('\n')

making it jade ready


In [21]:
print(dirstart+"optim-"+optimizer+appendname+model+dataset+".sh")

jade_scripts/optim-sgdPreResNet110CIFAR10.sh
